In [7]:
import easyocr
import cv2
import numpy as np

# Initialize EasyOCR reader
reader = easyocr.Reader(['en','bn'])  # Specify the languages you need

# Read the image
image_path = 'Example Images/test-3.PNG'
image = cv2.imread(image_path)

# Perform OCR on the image
results = reader.readtext(image_path, detail=1)

# Function to calculate the vertical center of a bounding box
def vertical_center(bbox):
    return (bbox[0][1] + bbox[2][1]) / 2

# Function to merge words into lines based on y-coordinate proximity
def merge_words_into_lines(results):
    lines = []
    current_line = []
    previous_y = None

    for result in results:
        bbox, text, prob = result
        current_y = vertical_center(bbox)

        if previous_y is None:
            current_line.append((text, bbox, prob))
        else:
            # If the current word is vertically close to the previous word, append to the current line
            if abs(current_y - previous_y) < 20:  # Adjust this threshold as needed
                current_line.append((text, bbox, prob))
            else:
                # Otherwise, start a new line
                lines.append(current_line)
                current_line = [(text, bbox, prob)]

        previous_y = current_y

    # Add the last line
    if current_line:
        lines.append(current_line)

    return lines

# Get the lines from the OCR results
lines = merge_words_into_lines(results)

# Store and print the bounding boxes and probabilities of each whole line
line_bboxes_probs = []

# Draw lines on the image
for line in lines:
    line_text = ' '.join([text for text, bbox, prob in line])
    print(f"Extracted Line: {line_text}")  # Print the line text

    # Calculate a bounding box around the whole line
    line_bboxes_coords = [bbox for text, bbox, prob in line]
    line_top_left = [min(bbox[0][0] for bbox in line_bboxes_coords), min(bbox[0][1] for bbox in line_bboxes_coords)]
    line_bottom_right = [max(bbox[2][0] for bbox in line_bboxes_coords), max(bbox[2][1] for bbox in line_bboxes_coords)]
    
    # Calculate the average probability for the line
    avg_prob = np.mean([prob for text, bbox, prob in line])
    
    # Store the bounding box and probability for the whole line as lists
    line_bbox_prob = [line_top_left, line_bottom_right, avg_prob]
    line_bboxes_probs.append(line_bbox_prob)

    # Draw a bounding box around the whole line
    cv2.rectangle(image, tuple(line_top_left), tuple(line_bottom_right), (255, 0, 0), 2)

# Print the stored bounding boxes and probabilities
for i, bbox_prob in enumerate(line_bboxes_probs):
    print(f"Line {i + 1} Bounding Box: {bbox_prob[:2]}, Average Probability: {bbox_prob[2]}")

# Save the output image
output_image_path = 'output_image_with_lines.jpg'
cv2.imwrite(output_image_path, image)

# Display the output image
cv2.imshow('Extracted Lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Extracted Line: Nari Pagol
Extracted Line: কত কাম করবি
Extracted Line: সব ধরসে একুরেট
Extracted Line: Nari Pagol
Extracted Line: ধরে টা কি
Extracted Line: ধরব তোরা কারে সাইবার বুলি করসস
Extracted Line: TreasurerN ihal repliEd to yCu
Extracted Line: মামা তোরা একটু বাংলা তে কথা বল প্লিজা আমি একটু টেস্ট করি]
Extracted Line: মিউমিউমিউ
Extracted Line: আগে টেক্সট বাইর করব স্ক্রিনশট থাইক্কা
Line 1 Bounding Box: [[51, 25], [111, 41]], Average Probability: 0.9405426283051362
Line 2 Bounding Box: [[51, 46], [152, 70]], Average Probability: 0.7950850348922183
Line 3 Bounding Box: [[999, 88], [1117, 117]], Average Probability: 0.8647988620966613
Line 4 Bounding Box: [[51, 135], [111, 151]], Average Probability: 0.9926682600486635
Line 5 Bounding Box: [[49, 157], [117, 177]], Average Probability: 0.6434916399029107
Line 6 Bounding Box: [[882, 198], [1116, 222]], Average Probability: 0.5556010785821337
Line 7 Bounding Box: [[55, 247], [235, 263]], Average Probability: 0.194899378886072
Line 8 Boundi